# M1.4 - Accéder aux données MERRA-2 dans le cloud

*Partie de:* [**Open Climate Data**](https://github.com/OpenClimateScience/M1-Open-Climate-Data) | **Leçon précédente** | **Leçon suivante**

**Contenu :**

- [Utilisation de `earthaccess`](#Utilisation-de-earthaccess)
- [Obtenir une série temporelle de température](#Obtenir-une-série-temporelle-de-température)

In [ ]:
import earthaccess
import xarray as xr
from matplotlib import pyplot

auth = earthaccess.login()

## Utilisation de `earthaccess`

Précédemment, nous avons téléchargé manuellement un fichier netCDF4 depuis NASA Earthdata Search. Maintenant, voyons comment nous pouvons utiliser du code Python pour télécharger des données depuis le cloud.

La bibliothèque `earthaccess` peut être utilisée pour rechercher et télécharger des données depuis Earthdata Search sans ouvrir de navigateur web ni cliquer un peu partout. Pour certaines applications, cela nous permet d'écrire du code Python plus transparent et reproductible, car cela permet à une autre personne d'obtenir facilement les mêmes données brutes avec lesquelles nous commençons.

Nous utilisons `earthaccess.search_data()` pour obtenir un ou plusieurs **granules** qui correspondent à une requête de recherche. Un granule est un ensemble de données unique associé à une mission ou un produit de la NASA.

**Vous vous souvenez où se trouve le nom court des produits Earthdata ?** Dans la page d'information (i), il y a deux emplacements :

![](./assets/M1_screenshot_Earthdata_Search_MERRA2_info.png)

**Nous utilisons le `short_name` (nom court) pour indiquer quelle collection de granules de données nous voulons rechercher.**

Dans cet exemple, nous trouvons qu'il y a 32 granules entre le 1er mai et le 1er juin inclus.

In [ ]:
results = earthaccess.search_data(
    short_name = 'M2SDNXSLV',
    temporal = ("2023-05", "2023-06"))
results[0]

Aucune donnée n'a encore été téléchargée ; nous avons juste une référence aux données stockées dans le cloud.

In [ ]:
type(results[0])

Pour télécharger un granule, nous pouvons utiliser `earthaccess.open()`. 

#### &#x1F6A9; <span style="color:red">Attention</red>

**Un point important à noter concernant `earthaccess.open()` est qu'il nécessite une *séquence* de granules à ouvrir.** Donc, même si nous voulons ouvrir un seul granule, celui-ci doit être fourni dans une liste ou un tuple Python.

In [ ]:
# REMARQUE : open() nécessite une séquence de références de fichiers
files = earthaccess.open(results[0:2])

In [ ]:
type(files[0])

Les granules ont été téléchargés dans la mémoire de notre ordinateur. Pour ouvrir l'un des fichiers téléchargés dans Python, nous utilisons maintenant `xarray`. Il y a un léger décalage lors de l'utilisation de `open_dataset()` sur un granule téléchargé via `earthaccess.open()`, car `xarray` doit analyser le fichier et comprendre le système de coordonnées et les dimensions.

In [ ]:
ds2 = xr.open_dataset(files[0])
ds2

Le `xarray.Dataset` résultant est prêt à être tracé !

In [ ]:
ds2['T2MMIN'].plot()

## Obtenir une série temporelle de température

Cela a bien fonctionné pour un seul fichier, mais que faire si nous voulions générer une série temporelle de données climatiques ? Nous savons que les jeux de données `xarray` peuvent avoir une dimension temporelle, ce qui les rend capables de représenter plusieurs moments dans le temps. Comment créer un tel ensemble de données ?

Dans cet exemple suivant, nous utilisons une boucle `for` pour itérer sur les granules MERRA-2, en ouvrant chacun d'eux, puis en sélectionnant la valeur `T2MMIN` (température minimale) à un emplacement spécifique. Nous ajoutons cette valeur à une liste, ainsi que la date (`"time"`) du granule, pour construire un ensemble de données de série temporelle.

In [ ]:
# REMARQUE : Cet exemple peut prendre une demi-minute avec une bonne connexion internet.
results = earthaccess.search_data(
    short_name = 'M2SDNXSLV',
    temporal = ("2023-05", "2023-06"))

time_list = []
data_list = []
file_list = earthaccess.open(results)
for filename in file_list:
    ds = xr.open_dataset(filename)
    data_list.append(ds['T2MMIN'].sel(lat = 36.5, lon = 3.125).values)
    time_list.append(ds['T2MMIN']['time'].values)

Nous avons maintenant une `liste` Python de valeurs de température minimale et une autre `liste` de dates. Ci-dessous, nous traçons ces données, après avoir utilisé `numpy` pour convertir les données de température de degrés Kelvin (K) à degrés Celsius (C).

In [ ]:
import numpy as np

# Convertir de degrés K à degrés C
data = np.array(data_list).ravel() - 273.15
time = np.array(time_list).ravel()

pyplot.figure(figsize = (12, 4))
pyplot.plot(time, data)
pyplot.ylabel('Température Min. (deg C)')